In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [2]:
full = pd.concat([train, test], ignore_index=True)

In [41]:
print(train.shape)
print(test.shape)
print(full.shape)
full.head()

(550068, 12)
(233599, 11)
(783667, 17)


,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,AmountBy_User,AmountBy_Product,Product_Category_1_Amount,Product_Category_2_Amount,Product_Category_3_Amount
0,1,0,0,0,10,3,0.0,0.0,684,8370.0,2,0,334093.0,2694686.0,119859.0,0.0,0.0
1,1,0,0,0,10,1,6.0,14.0,2406,15200.0,2,0,334093.0,9472642.0,61604.0,31822.0,27268.0
2,1,0,0,0,10,12,0.0,0.0,868,1422.0,2,0,334093.0,126265.0,61279.0,0.0,0.0
3,1,0,0,0,10,12,14.0,0.0,844,1057.0,2,0,334093.0,496203.0,61279.0,27268.0,0.0
4,8,2,1,0,16,8,0.0,0.0,2769,7969.0,4,1,810472.0,1561631.0,460149.0,0.0,0.0


In [3]:
# Monto por Usuario
Amount_byUser = pd.DataFrame(full.groupby(['User_ID'])['Purchase'].sum())
Amount_byUser.columns = ['AmountBy_User']
Amount_byUser.reset_index(level=0, inplace=True)

# Monto por producto
Amount_byProduct = pd.DataFrame(full.groupby(['Product_ID'])['Purchase'].sum())
Amount_byProduct.columns = ['AmountBy_Product']
Amount_byProduct.reset_index(level=0, inplace=True)

# Montos por Categoria
Amount_CU1 = pd.DataFrame(full.groupby(['User_ID','Product_Category_1'])['Purchase'].sum())
Amount_CU1.reset_index(level=['User_ID','Product_Category_1'], inplace=True)
Amount_CU1.columns = ['User_ID','Product_Category','Amount1']

Amount_CU2 = pd.DataFrame(full.groupby(['User_ID','Product_Category_2'])['Purchase'].sum())
Amount_CU2.reset_index(level=['User_ID','Product_Category_2'], inplace=True)
Amount_CU2.columns = ['User_ID','Product_Category','Amount2']

Amount_CU3 = pd.DataFrame(full.groupby(['User_ID','Product_Category_3'])['Purchase'].sum())
Amount_CU3.reset_index(level=['User_ID','Product_Category_3'], inplace=True)
Amount_CU3.columns = ['User_ID','Product_Category','Amount3']

Amount_CU = Amount_CU1.merge(Amount_CU2, how="outer").merge(Amount_CU3, how="outer")
Amount_CU.fillna(0,inplace = True)
Amount_CU['Category_Amount'] = Amount_CU.Amount1 + Amount_CU.Amount2 + Amount_CU.Amount3
Amount_CU.drop(['Amount1','Amount2','Amount3'], axis = 1 ,inplace=True)

In [5]:
# Cruzamos la info anterior
full = full.merge(Amount_byUser, how = "left").merge(Amount_byProduct, how = "left")

full = pd.merge(full, Amount_CU, how="left", left_on = ['User_ID','Product_Category_1'], right_on = ['User_ID','Product_Category'])
full.rename(columns={'Category_Amount':'Product_Category_1_Amount'}, inplace=True)
                
full = pd.merge(full, Amount_CU, how="left", left_on = ['User_ID','Product_Category_2'], right_on = ['User_ID','Product_Category'])
full.rename(columns={'Category_Amount':'Product_Category_2_Amount'}, inplace=True)

full = pd.merge(full, Amount_CU, how="left", left_on = ['User_ID','Product_Category_3'], right_on = ['User_ID','Product_Category'])
full.rename(columns={'Category_Amount':'Product_Category_3_Amount'}, inplace=True)

full.drop(['Product_Category_x','Product_Category_y','Product_Category'], axis = 1 ,inplace=True)

# Completamos los 0
full['Product_Category_1'].fillna(0,inplace = True)
full['Product_Category_2'].fillna(0,inplace = True)
full['Product_Category_3'].fillna(0,inplace = True)
full['Product_Category_1_Amount'].fillna(0,inplace = True)
full['Product_Category_2_Amount'].fillna(0,inplace = True)
full['Product_Category_3_Amount'].fillna(0,inplace = True)

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,AmountBy_User,AmountBy_Product,Product_Category_1_Amount,Product_Category_2_Amount,Product_Category_3_Amount
0,0-17,A,F,0,10,3,0.0,0.0,P00069042,8370.0,2,1000001,334093.0,2694686.0,119859.0,NaN,NaN
1,0-17,A,F,0,10,1,6.0,14.0,P00248942,15200.0,2,1000001,334093.0,9472642.0,61604.0,31822.0,27268.0
2,0-17,A,F,0,10,12,0.0,0.0,P00087842,1422.0,2,1000001,334093.0,126265.0,61279.0,NaN,NaN
3,0-17,A,F,0,10,12,14.0,0.0,P00085442,1057.0,2,1000001,334093.0,496203.0,61279.0,27268.0,NaN
4,55+,C,M,0,16,8,0.0,0.0,P00285442,7969.0,4+,1000002,810472.0,1561631.0,460149.0,NaN,NaN


In [6]:
# mapeamos informacion
mapping_age = {'0-17': 1, '18-25': 2, '26-35': 3, '36-45' : 4, '46-50'  : 5, '51-55':6, '55+' : 7}
mapping_siccy = {'1': 1, '2': 2, '3': 3, '4+': 4, '0': 0}

# Transformamos strings
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
full.City_Category = le.fit_transform(full.City_Category)
full.Gender = le.fit_transform(full.Gender)
full.Product_ID = le.fit_transform(full.Product_ID)
full.User_ID = le.fit_transform(full.User_ID)
full.replace({'Age': mapping_age, 'Stay_In_Current_City_Years': mapping_siccy}, inplace = True)
full.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,AmountBy_User,AmountBy_Product,Product_Category_1_Amount,Product_Category_2_Amount,Product_Category_3_Amount
0,1,0,0,0,10,3,0.0,0.0,684,8370.0,2,0,334093.0,2694686.0,119859.0,0.0,0.0
1,1,0,0,0,10,1,6.0,14.0,2406,15200.0,2,0,334093.0,9472642.0,61604.0,31822.0,27268.0
2,1,0,0,0,10,12,0.0,0.0,868,1422.0,2,0,334093.0,126265.0,61279.0,0.0,0.0
3,1,0,0,0,10,12,14.0,0.0,844,1057.0,2,0,334093.0,496203.0,61279.0,27268.0,0.0
4,7,2,1,0,16,8,0.0,0.0,2769,7969.0,4,1,810472.0,1561631.0,460149.0,0.0,0.0


In [7]:
import os
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.2.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

def get_model(estimator, parameters, X_train, y_train, scoring):  
    model = GridSearchCV(estimator, param_grid=parameters, scoring=scoring)
    model.fit(X_train, y_train)
    return model.best_estimator_

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

clf1 = LinearRegression()
clf2 = Lasso(alpha = 0.01)
clf3 = RandomForestRegressor(max_features = 10, max_depth = 18, n_estimators = 60)
clf4 = xgb.XGBRegressor()
clf5 = KNeighborsRegressor(n_neighbors = 3)
#clf6 = SVR()

n_train = full[~full.Purchase.isnull()]
n_test = full[full.Purchase.isnull()]

y_train = np.asarray(n_train.Purchase)
X_train = n_train.drop('Purchase',axis=1)


res_reglin,res_rf,res_xgb, res_lasso,res_kn, res_svc = [],[],[],[],[],[]

for i in range(9,11):
    res_reglin = np.append(res_reglin,cross_val_score(clf1, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    res_lasso = np.append(res_lasso,cross_val_score(clf2, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    res_rf = np.append(res_rf,cross_val_score(clf3, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    res_xgb = np.append(res_xgb,cross_val_score(clf4, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    res_kn = np.append(res_kn,cross_val_score(clf5, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    #res_svc = np.append(res_kn,cross_val_score(clf6, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))

print(pd.Series(-res_reglin).describe())
print(pd.Series(-res_lasso).describe())
print(pd.Series(-res_rf).describe())
print(pd.Series(-res_xgb).describe())
print(pd.Series(-res_kn).describe())
#print(pd.Series(-res_svc).describe())

KeyboardInterrupt: 

In [17]:
rf = RandomForestRegressor()
scoring = make_scorer(mean_absolute_error, greater_is_better=False)
parameters = {'max_features' : [10], 'max_depth' : [18], 'n_estimators' : [60]}
reg_rf = get_model(rf, parameters, X_train, y_train, scoring)
reg_rf

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=18,
           max_features=10, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
XGB = xgb.XGBRegressor(seed= 42)
scoring = make_scorer(mean_absolute_error, greater_is_better=False)
parameters = {'max_depth':np.arange(2,8), 'learning_rate': np.linspace(0.01,0.3,10),'reg_alpha':np.linspace(0.1,1.0,5), 'reg_lambda': np.linspace(1.0,3.0,5)}
reg_xgb = get_model(XGB, parameters, X_train, y_train, scoring)
reg_xgb

In [30]:
clf3 = RandomForestRegressor(max_features = 10, max_depth = 18, n_estimators = 60)
clf3.fit(X_train, y_train)
pred = clf3.predict(n_test.drop('Purchase',axis=1))

submission = pd.DataFrame({
        "User_ID": test.User_ID,
        "Product_ID": test.Product_ID,
        "Purchase": pred
    }, columns = ["User_ID","Product_ID","Purchase"])
submission.to_csv('prediction_solutionv1.csv', index=False)

In [29]:
n_test.isnull().sum()

Age                                0
City_Category                      0
Gender                             0
Marital_Status                     0
Occupation                         0
Product_Category_1                 0
Product_Category_2                 0
Product_Category_3                 0
Product_ID                         0
Purchase                      233599
Stay_In_Current_City_Years         0
User_ID                            0
AmountBy_User                      0
AmountBy_Product                   0
Product_Category_1_Amount          0
Product_Category_2_Amount          0
Product_Category_3_Amount          0
dtype: int64